# Comet Test Notebook 

In [ ]:
from comet_ml import Experiment
experiment = Experiment(api_key="zNkJjcVKOMD5gKd05z6CwT4OD",
                        project_name="team-rm5-sigmoidfreuds", workspace="lizette95")

In [ ]:
# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

# Prerequisites
#import sys
#!{sys.executable} -m pip install wordcloud comet_ml scikit-learn

# Load Dependencies
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, accuracy_score,precision_score, recall_score, classification_report, confusion_matrix

In [ ]:
train_data = pd.read_csv('https://raw.githubusercontent.com/Lizette95/classification-predict-streamlit-template/master/Data/train.csv')

In [ ]:
# contractions = {"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have", "i'm": "I am", "i've": "I have", "isn't": "is not", "it'd": "it had", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there had", "there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we had", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'alls": "you alls", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you had", "you'd've": "you would have", "you'll": "you you will", "you'll've": "you you will have", "you're": "you are", "you've": "you have"}
# contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
# def expand_contractions(text):
#     """
#     Expand contractions using dictionary of contractions.
#     """
#     text = text.lower()
#     def replace(match):
#         return contractions[match.group(0)]
#     return contractions_re.sub(replace, text)

# train_data['message'] = train_data['message'].apply(expand_contractions)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['via','rt'])

def remove_noise(tweet):
    """
    Remove noise from text data, such as newlines, punctuation, URLs and numbers.
    """
    new_tweet = BeautifulSoup(tweet, "lxml").text #HTML Decoding
    #new_tweet = re.sub(r'http\S+', 'url', new_tweet) #Replace URLs
    new_tweet = new_tweet.lower()
    new_tweet = new_tweet.replace('\n',' ') #Remove Newlines
    new_tweet = re.sub('[/(){}\[\]\|@,;]',' ', new_tweet)
    new_tweet = re.sub('[^0-9a-z #+_]', '', new_tweet) #Remove Symbols
    new_tweet = ' '.join(word for word in new_tweet.split() if word not in stop_words)
    return new_tweet

train_data['message'] = train_data['message'].apply(remove_noise)

In [ ]:
# train_data = train_data.drop_duplicates(subset='message', keep='first')

In [ ]:
X = train_data.message
y = train_data.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
print("\nResults\nConfusion matrix \n {}".format(confusion_matrix(y_test, y_pred)))
f1 = f1_score(y_test, y_pred,average="macro")
precision = precision_score(y_test, y_pred,average="macro")
recall = recall_score(y_test, y_pred,average="macro")
params = {"random_state": 42,
          "model_type": "logreg",
          "scaler": "standard scaler",
          #"param_grid": "str(param_grid)",
          "stratify": True,
          }
metrics = {"f1": f1,
           "recall": recall,
           "precision": precision
           }

In [ ]:
experiment.log_parameters(params)
experiment.log_metrics(metrics)

In [ ]:
experiment.end()

In [ ]:
experiment.display()

End.